In [1]:
import pandas as pd
import numpy as np
import random
from ortools.sat.python import cp_model
import names

In [2]:
def doub_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2])

def tri_row(row):
    hora_inicial = pd.to_datetime(row['Horário'], format='%H:%M:%S')
    
    nova_linha_1 = row.copy()
    nova_linha_2 = row.copy()
    nova_linha_3 = row.copy()
    
    nova_linha_1['Horário'] = (hora_inicial).strftime('%H:%M:%S')
    nova_linha_2['Horário'] = (hora_inicial + pd.Timedelta(hours=1)).strftime('%H:%M:%S')
    nova_linha_3['Horário'] = (hora_inicial + pd.Timedelta(hours=2)).strftime('%H:%M:%S')
    
    return pd.DataFrame([nova_linha_1, nova_linha_2, nova_linha_3])

def desaninhar_dias(df):
    df['Dias da Semana'] = df['Dias da Semana'].str.split(',')
    df = df.explode('Dias da Semana')
    return df

# Aplicar a função para todas as linhas triplas
def apply_func(df,func):
    linhas_triplicadas = df.apply(func, axis=1)
    return pd.concat(linhas_triplicadas.tolist(), ignore_index=True)

In [3]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA TESTE CARLOS E ALLAN.xlsx',header=1)
aulas_tratadas = aulas_raw.loc[~aulas_raw['Grupo'].isnull()][['Grupo','Horário','Dias da Semana']]

In [4]:
aulas_tratadas = aulas_tratadas.loc[aulas_tratadas['Dias da Semana']!='FOLDER'].reset_index(drop=True)

In [5]:
aulas = aulas_tratadas.copy()
# tratando os dados para colocar cada linha uma aula
aulas['Dias da Semana'] = aulas['Dias da Semana'].str.replace('EVERYDAY','2ª ● 3ª ● 4ª ● 5ª ● 6ª')

# separando as aulas que são triplas, duplas e o resto
tri = aulas.loc[aulas['Dias da Semana'] == 'Saturday - Triple']
doub = aulas.loc[aulas['Dias da Semana'].str.contains('DOUBLE')]
aulas_simples = aulas[~aulas['Dias da Semana'].str.contains('DOUBLE|Saturday - Triple')].copy()

# tratando a colunas horario
aulas_simples['Horário'] = pd.to_datetime(aulas_simples['Horário'],format='%H:%M:%S').dt.strftime('%H:%M:%S')

In [6]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

In [7]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

# desaninhando os dias da semana
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('●',',')

In [8]:
# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = apply_func(doub, doub_row)
aulas_triplicadas = apply_func(tri, tri_row)

# padronizando a coluna de dias da semana
aulas_duplicadas['Dias da Semana'] = aulas_duplicadas['Dias da Semana'].str.replace('DOUBLE','')
aulas_triplicadas['Dias da Semana'] = aulas_triplicadas['Dias da Semana'].str.replace(' - Triple','')

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

# desaninhando os dias da semana
df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('●',',').str.replace(' ','').str.split(',')
df_tratado = df_tratado.explode('Dias da Semana').reset_index(drop=True)

df_tratado['Dias da Semana'] = df_tratado['Dias da Semana'].str.replace('ª','ª,').str.split(',')
df_tratado = df_tratado.explode('Dias da Semana').reset_index(drop=True)

df_tratado = df_tratado[df_tratado['Dias da Semana']!=''].copy()


In [9]:
nomes_distintos = set()  # Usar um conjunto para garantir a distinção

while len(nomes_distintos) < 147:
    primeiro_nome = names.get_first_name()
    nomes_distintos.add(primeiro_nome)

professors = list(nomes_distintos)
len(professors)

147

variaveis sem horario e dia da semana (teste)

In [10]:
# Dicionário de alocações
alocacoes = {}

# Modelo
model = cp_model.CpModel()

for i in professors:
    for g in df_tratado['Grupo'].unique():
        alocacoes[(i,g)] = model.NewBoolVar(f"{i}_converinglesson_{g}")

### condição : só um professor por grupo

In [11]:
for g in df_tratado['Grupo'].unique():
    model.Add(sum(alocacoes[(i,g)] for i in professors) == 1)

### condição : professor só da aula em uma turma de mesmo horario

In [12]:
horarios = df_tratado['Horário'].unique()
for h in horarios:
    grupos_no_mesmo_horario = df_tratado.loc[df_tratado['Horário'] == h]['Grupo'].unique()
    for i in professors:
        model.Add(sum(alocacoes[(i,g)] for g in grupos_no_mesmo_horario) <= 1)

In [13]:
solver = cp_model.CpSolver()
status = solver.Solve(model)
prof_alocados = pd.DataFrame(columns=['Professor', 'Grupo'])

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for g in df_tratado['Grupo'].unique():
        for i in professors:
            if solver.Value(alocacoes[(i, g)]):
                print(f"O professor {i} foi alocado ao grupo {g}")
                aloca = pd.DataFrame({'Professor': [i], 'Grupo': [g]})
                prof_alocados = pd.concat([prof_alocados, aloca], ignore_index=True)

else:
    print("Nenhuma solução viável foi encontrada.")

O professor John foi alocado ao grupo MBA DAVI BRAGA
O professor Bertha foi alocado ao grupo MBA LIGA DOS CAMPEÕES
O professor Bertha foi alocado ao grupo MBA IDEIAS EM AÇÃO
O professor Bertha foi alocado ao grupo GREENSBURG ONLINE
O professor Renee foi alocado ao grupo SAVANAH PRESENCIAL
O professor Bertha foi alocado ao grupo CONVERSATION 14 PRESENCIAL
O professor Anne foi alocado ao grupo FERNANDO CAJE - PRESENCIAL
O professor John foi alocado ao grupo KNOXVILLE PRESENCIAL
O professor John foi alocado ao grupo GREAT BRITAIN PRESENCIAL
O professor Anne foi alocado ao grupo GARLAND PRESENCIAL
O professor Heather foi alocado ao grupo TAMPA PRESENCIAL
O professor Bertha foi alocado ao grupo CAPE COD PRESENCIAL
O professor Linda foi alocado ao grupo TABLEDERRY ONLINE
O professor Anne foi alocado ao grupo ANA PAULA - ONLINE
O professor John foi alocado ao grupo KENTUCKY ONLINE
O professor Derek foi alocado ao grupo CURAÇAU ONLINE
O professor Heather foi alocado ao grupo KIKO - VIP - PRESE

In [14]:
aulas_finais = aulas_tratadas.merge(prof_alocados, on='Grupo', how='left')

In [15]:
aulas_finais

,Grupo,Horário,Dias da Semana,Professor
0,MBA DAVI BRAGA,14:00:00,2ª,John
1,MBA LIGA DOS CAMPEÕES,18:30:00,5ª,Bertha
2,MBA IDEIAS EM AÇÃO,19:00:00,2ª,Bertha
3,GREENSBURG ONLINE,07:00:00,2ª ● 3ª ● 4ª ● 5ª,Bertha
4,INTENSIVÃO 3 ONLINE,19:00:00,DOUBLE EVERYDAY,Adam
...,...,...,...,...
142,AUCKLAND PRESENCIAL,19:00:00,2ª ● 3ª ● 4ª ● 5ª,Diana
143,QUEBEC PRESENCIAL,20:00:00,2ª ● 4ª ● 5ª,Erica
144,GUADALAJARA ONLINE,19:00:00,2ª ● 3ª ● 4ª ● 5ª,Shirley
145,VALENCIA - ONLINE,20:00:00,2ª ● 3ª ● 4ª ● 5ª,Jimmie
